### TapTube Popularity Channel Predict

<center> <img src="images/potarataptube.jpg"  /> </center>

### Abstract

In questo tutorial, costruiremo un sistema di predizione in tempo reale per valutare la popolarità di un canale YouTube basandoci su vari parametri, che verranno descritti nella sezione ***Pipeline*** in particolare nella sottosezione ***Producer*** . Questo ci permetterà di distinguere tra canali attivi e inattivi, nonché tra creatori di contenuti e non.

### Data Flow

<center> <img src="images/struttura.png"  /> </center>

### Pipeline
Le tecnologie utilizzate in questo progetto sono:
1. Uno script in Python per raccogliere i dati relativi a YouTube: Producer
2. Uno strumento per l'ingestion dei dati: Logstash.
3. Uno strumento per lo streaming dei dati: Apache Kafka.
4. Uno strumento per il processamento dei dati: Spark Structured Streaming (Apache Spark) con il supporto di **MLlib** per l'analisi e il machine learnine e **Databricks** per testing
5. Uno strumento per l'indicizzazione dei dati: Elasticsearch.
6. Uno strumento per la visualizzazione dei dati: Kibana.

#### Producer

Nel mio progetto, ho utilizzato un script denominato tubescraper.py per interagire con l'API pubblica di YouTube (allego il link https://developers.google.com/youtube/v3?hl=it) e ottenere i dati necessari. Questa API consente di accedere a una vasta gamma di informazioni sui canali, tuttavia, è importante notare che c'è un limite massimo di 10.000 query al giorno per ogni chiave API, il che impone una restrizione sul numero di richieste che possono essere effettuate (il limite può essere cambiato ma ai fini del progetto sono rimasto entro le 10000 query al giorno).

Nel mio script, ho estratto i seguenti parametri chiave per ogni canale:

- ChannelID: l'ID unico del canale, ottenuto dal campo channel['channelId'].
- Title: il titolo del canale.
- Country: il paese di origine del canale.
- Subscribers: il numero di iscritti al canale.
- TotalVideo: il numero totale di video caricati sul canale.
- Views: il numero totale di visualizzazioni che i video del canale hanno accumulato.
- Join_date: la data in cui il canale è stato creato.

Da notare che i parametri <i> Subscribers </i> e <i> Views </i> sono stati utilizzati per la predizione della popolarità del canale.

<center> <img src="images/stonks.jpg"  /> </center>

#### Logstash

#### Apache Kafka (https://kafka.apache.org/)

<center> <img src="images/kafka.jpg"  /> </center>

Apache Kafka è una piattaforma distribuita basata su messaggi e sul modello publish-subscribe, facilita la scalabilità e l'affidabilità nell'invio e nella ricezione di messaggi tra applicazioni, fornendo una piattaforma robusta per l'integrazione e l'elaborazione dei dati in tempo reale.

Concetti chiave di Kafka:

- **Producer**: Chi invia i dati al sistema Kafka. Il producer determina su quale topic inviare il messaggio.

- **Topic**: Rappresenta l'argomento a cui vengono inviati i messaggi dai producer e da cui i consumer leggono i messaggi. È l'unità fondamentale di categorizzazione dei dati.

- **Partizione**: È una struttura di dati all'interno di un topic in cui vengono scritti i messaggi. Ogni topic può essere suddiviso in una o più partizioni, consentendo una distribuzione dei dati scalabile e parallela.

- **Consumer**: Chi legge i messaggi da uno o più topic. Ogni consumer appartiene a un gruppo di consumer, che cooperano per consumare i messaggi in modo distribuito e parallelo.

<center> <img src="images/struttura_kafka.png" width=800 height=500 /> </center>

Nel nostro caso:
- **Producer**: Logstash
- **Topic**: taptube_channel
- **Consumer**: Apache Spark

Inoltre kafka, all'interno del nostro progetto, è cosi suddiviso:

- **Zookeper**: servizio per la sincronizzazione dei consumer;
- **Broker**: Cuore di Kafka, gestisce il flusso di dati e mantiene storati i messaggi in attesa dei consumatori.
- **Init-Kafka**: Script per l'inizializzazione di topic, partition..
- **Kafka-UI**: interfaccia Web relativa a Kafka.


#### Apache Spark (https://spark.apache.org/) 

<center> <img src="images/spark.png" /> </center>

Apache Spark è un framework open-source progettato per il calcolo distribuito e il trattamento di dati su larga scala. È progettato per essere veloce, scalabile e offre un'interfaccia unificata per il trattamento di dati sia batch che in tempo reale.

<center> <img src="images/struttura_spark.png"  /> </center>

##### Dataset

Per allenare il modello si è scelto di utilizzare il dataset fornito da Kaggle (https://www.kaggle.com/datasets/asaniczka/2024-youtube-channels-1-million/data), però è stato necessario affrontare diversi passaggi per preparare i dati in modo appropriato per l'addestramento e l'elaborazione in tempo reale tramite Kafka; rimuovere NaN, gestire gli outliers, ridurre le dimensioni del dataset, rimuovere le features non importanti ne sono alcuni esempi per rendere il nostro dataset più ***light***

##### Databriks
Databricks è una piattaforma di analisi dei dati basata su Apache Spark, progettata per semplificare lo sviluppo, la gestione e il monitoraggio di applicazioni di big data e machine learning.

Ho utilizzato Databriks (la Community Edition) per testare il dataset con la quale allenare il modello prima di dockerizzare il tutto e non far implodiere il mio pc, ma c'è stato un piccolo problema con la grandezza del dataset.

<center> <img src="images/databriks.jpg" /> </center>

<center> Risorse limitate.... </center>

##### Training


Per l'allenamento del modello è stato utilizzato PySpark ed in particolare la libreria Spark MLlib utilizzando la funzione "KMeans" all'interno di una pipeline di machine learning. 

Quando avviene la chiamata pipeline.fit(), viene adattato il modello al training set di input e gli stage vengono eseguiti in ordine per ogni record del dataset

<center> <img src="images/training.jpg" /> </center>

 Il modello prodotto ha ottenuto un'accuracy del 80%  circa ed in generale delle buone performance.

##### Streaming

Allenato il modello si è passati all'utilizzo di Spark Structured Streaming, che ci permette di lavorare su dati in tempo reale, integrandosi perfettamente con Kafka.

Ogni dato ricevuto, viene passato al modello, il quale effettua la predizione.

I dati predetti vengono poi depositati su elasticsearch e quindi visualizzati con kibana.


trained model
prediction = model.transform(dataset) 

#### Visualization: ElasticSearch + Kibana

AHHH finalmente, il tutto è stato configurato ora basta che passo tutto a kibana con elestic

<center> <img src="images/dockertoresource.jpg" width=600 height=400 /> </center>

<center> <img src="images/elastickibana_2.png" width=700 height=200 /> </center>

**Elasticsearch** è un motore per la ricerca e l'analisi di dati: è in grado di gestire tutte le tipologie di dato (testuale, numerico, geospaziale, strutturato e non strutturato) ed è conosciuto per la sua natura distribuita, velocità e scalabilità.

Elasticsearch è stato utilizzato per indicizzare e memorizzare i dati precedentemente elaborati e inviarli a Kibana per visualizzarli. 

**Kibana** è una UI utilizzata per visualizzare ed analizzare i dati di Elasticsearch con l'ausilio di grafici.

Di seguito vengono riportati alcuni grafici mostrati nella dashboard. 

<center> <img src="images/ciaoo.jpg" /> </center>